In [26]:
import detectron2
import torch
import copy
import pickle 
from PIL import Image
from detectron2.data import DatasetCatalog
from detectron2.data import MetadataCatalog
import detectron2.data.transforms as T
from detectron2.data import DatasetMapper   # the default mapper
from detectron2.data import build_detection_train_loader
from detectron2.data import detection_utils as utils 
from detectron2.data.detection_utils import annotations_to_instances

In [2]:
torch.__version__

'1.10.0'

# Register the dataset so that it can be used by dataloader

In [3]:
def get_detectron_data():
    data = [] #here is where we will collect the coco dicts
    root = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/SCC-Tumor-Detection/Gokul_files/Nuclei_Detection/Data/DetectronData/"
    xml_files = [
        "327_A1d_ASAP.xml",
        "281_A1f_ASAP.xml",
        "270_A1d_ASAP.xml"
    ]
    for id in xml_files: #collect all of the individual WSI data 
        sample_id = id.split(".")[0]
        sample_data = None
        filepath = str(root + sample_id+".pickle")
        with open(filepath, 'rb') as handle:
            sample_data = pickle.load(handle)
        sample_data 
        for patch_info in sample_data:
            data.append(patch_info)
    return data


In [4]:
data = get_detectron_data()

In [5]:
len(data)

314

In [6]:
data[0]

{'file_name': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/SCC-Tumor-Detection/Gokul_files/Nuclei_Detection/Data/Images/327_A1d_ASAP_0.jpg',
 'height': 128,
 'width': 128,
 'image_id': '327_A1d_ASAP_0',
 'annotations': [{'bbox': [19166.5527, 18793.3867, 19183.4453, 18809.127],
   'bbox_mode': 0,
   'category_id': 1,
   'segmentation': [[19183.445299999992,
     18803.369099999996,
     19183.440791559675,
     18803.419837385467,
     19183.43585161297,
     18803.47057068638,
     19183.4304818525,
     18803.521297262872,
     19183.424683970905,
     18803.572014475085,
     19183.41845966082,
     18803.622719683164,
     19183.411810614878,
     18803.673410247244,
     19183.404738525718,
     18803.72408352748,
     19183.39724508597,
     18803.774736883996,
     19183.389331988266,
     18803.825367676945,
     19183.38100092525,
     18803.875973266466,
     19183.372253589532,
     18803.926551012697,
     19183.363091673782,
     18803.97709827

In [7]:
data[150]["annotations"] #[0]["segmentation"])

[{'bbox': [14686.9355, 7628.12207, 14709.8311, 7649.16162],
  'bbox_mode': 0,
  'category_id': 1,
  'segmentation': [[14703.024400000002,
    7628.122070000001,
    14703.09420295114,
    7628.13483953578,
    14703.163803127687,
    7628.148192623274,
    14703.233197591511,
    7628.162125808557,
    14703.302383404522,
    7628.176635637717,
    14703.371357628612,
    7628.191718656833,
    14703.440117325681,
    7628.20737141199,
    14703.508659557621,
    7628.223590449272,
    14703.576981386335,
    7628.240372314759,
    14703.645079873717,
    7628.257713554535,
    14703.712952081662,
    7628.275610714688,
    14703.780595072065,
    7628.294060341291,
    14703.848005906826,
    7628.313058980434,
    14703.915181647848,
    7628.332603178202,
    14703.982119357013,
    7628.35268948067,
    14704.048816096227,
    7628.373314433926,
    14704.11526892738,
    7628.39447458405,
    14704.18147491238,
    7628.41616647713,
    14704.247431113114,
    7628.438386659243,
 

In [8]:
isinstance(data[100]["annotations"][0]["segmentation"][0][1], float)

True

In [9]:
data[0]

{'file_name': '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/SCC-Tumor-Detection/Gokul_files/Nuclei_Detection/Data/Images/327_A1d_ASAP_0.jpg',
 'height': 128,
 'width': 128,
 'image_id': '327_A1d_ASAP_0',
 'annotations': [{'bbox': [19166.5527, 18793.3867, 19183.4453, 18809.127],
   'bbox_mode': 0,
   'category_id': 1,
   'segmentation': [[19183.445299999992,
     18803.369099999996,
     19183.440791559675,
     18803.419837385467,
     19183.43585161297,
     18803.47057068638,
     19183.4304818525,
     18803.521297262872,
     19183.424683970905,
     18803.572014475085,
     19183.41845966082,
     18803.622719683164,
     19183.411810614878,
     18803.673410247244,
     19183.404738525718,
     18803.72408352748,
     19183.39724508597,
     18803.774736883996,
     19183.389331988266,
     18803.825367676945,
     19183.38100092525,
     18803.875973266466,
     19183.372253589532,
     18803.926551012697,
     19183.363091673782,
     18803.97709827

In [10]:
DatasetCatalog.register("nuclei_detection", get_detectron_data)
# later, to access the data:
data = DatasetCatalog.get("nuclei_detection")

In [11]:
# MetadataCatalog.get("nuclei_detection").thing_classes = ["inflamm", "scc"] #0,1

In [12]:
# def mapper(dataset_dict):
#     dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
#     # can use other ways to read image
#     image = utils.read_image(dataset_dict["file_name"], format="BGR")
#     # See "Data Augmentation" tutorial for details usage
#     auginput = T.AugInput(image)
#     transform = T.Resize((128, 128))(auginput)
#     image = torch.from_numpy(auginput.image.transpose(2, 0, 1))
#     annos = [
#         utils.transform_instance_annotations(annotation, [transform], image.shape[1:])
#         for annotation in dataset_dict.pop("annotations")
#     ]
#     return {
#        # create the format that the model expects
#        "image": image,
#        "instances": utils.annotations_to_instances(annos, image.shape[1:])
#     }

In [31]:
import copy
import logging
import numpy as np
from typing import List, Optional, Union
import torch

from detectron2.config import configurable

from detectron2.data import detection_utils as utils 
import detectron2.data.transforms as T

In [35]:
dataloader = build_detection_train_loader(
    data,
    mapper=DatasetMapper(is_train=True,
                         augmentations=[T.Resize((128, 128))], 
                         use_instance_mask = True, 
                         instance_mask_format = "polygon",
                         image_format="BGR"), 
    total_batch_size = 1, 
    )

In [36]:
for x in dataloader:
    print(x[0]["instances"])
    break

Instances(num_instances=0, image_height=128, image_width=128, fields=[gt_boxes: Boxes(tensor([], size=(0, 4))), gt_classes: tensor([], dtype=torch.int64), gt_masks: PolygonMasks(num_instances=0)])


In [37]:
detectron2.data.detection_utils.annotations_to_instances(data[0]["annotations"], (128,128), "polygon")

Instances(num_instances=2, image_height=128, image_width=128, fields=[gt_boxes: Boxes(tensor([[19166.5527, 18793.3867, 19183.4453, 18809.1270],
        [19166.9355, 18781.1016, 19191.5059, 18797.2266]])), gt_classes: tensor([1, 1]), gt_masks: PolygonMasks(num_instances=2)])